In [1]:
import random
import pandas as pd
from Bio import pairwise2
from Bio.Align import substitution_matrices

In [6]:
#peptides ordered from SB tech
pep1 = '*********'
pep2 = '*********'

In [ ]:
scambledpeps = {}
for thepep in [pep1,pep2]:
    print(thepep)
    peplist = []
    while len(peplist) < 10:
        j = ''.join(random.sample(thepep,len(thepep)))
        np = ''.join(j)
        hammingdis = 0
        for p1,p2 in zip(thepep,np):
            if p1!=p2: hammingdis += 1
        #print(np,' hDis:',hammingdis)
        if hammingdis == len(thepep): 
            print('np',' hDis:',hammingdis)
            if np not in peplist:
                peplist.append(np)
    scambledpeps[thepep] = peplist

In [4]:
blosum62 = substitution_matrices.load("BLOSUM62")

In [5]:
scramb_df = pd.DataFrame()
for ii,x in enumerate(scambledpeps):
    sdf = pd.DataFrame(scambledpeps[x])
    sdf.columns = ['scrambled_pep']
    sdf['ref_pep'] = 10*[x]
    sdf['ref_name'] = 10*['TX{}'.format(ii+2)]
    sdf['hamming_dist'] = 10*[20]
    sim_score = []
    for i,d in sdf.iterrows():
        ref = d.ref_pep
        scram = d.scrambled_pep
        alignment = pairwise2.align.globalds(ref, scram, blosum62, -0.5, -0.05 ,score_only=True)
        sim_score.append(alignment)
    sdf['blosum62_sim_score'] = sim_score
    scramb_df = pd.concat([scramb_df,sdf])
scramb_df.sort_values(['ref_name','blosum62_sim_score'],inplace=True)

In [37]:
scamb_df.to_csv('Scambled_peptides_and_hammingDist.csv',sep=',')